# Hubble Catalog of Variables Notebook (CasJobs version)
### 2019 - 2022, Rick White, Steve Lubow, Trenton McKinney

This notebook shows how to access the [Hubble Catalogs of Variables (HCV)][1]. The HCV is a large catalog of faint variable objects extracted from version 3 of the [Hubble Source Catalog][2].  The HCV project at the National Observatory of Athens was funded by the European Space Agency (PI: Alceste Bonanos). The data products for the HCV are available both at the [ESA Hubble Archive][3] at [ESAC][4] through the [HCV Explorer][5] interface and at STScI.
See [Bonanos et al. (2019)][6] for more details.

Data tables in [MAST CasJobs][7] are queried from Python using the [mastcasjobs][8] module. For similar examples using the [MAST API][9], which is easier to use but less powerful than CasJobs, see [HCV_API_demo][10].


  [1]: http://archive.stsci.edu/hlsp/hcv/
  [2]: https://archive.stsci.edu/hst/hsc
  [3]: http://archives.esac.esa.int/ehst
  [4]: https://www.cosmos.esa.int/web/esdc
  [5]: http://archives.esac.esa.int/hcv-explorer
  [6]: https://ui.adsabs.harvard.edu/abs/2019A%26A...630A..92B/abstract
  [7]: https://mastweb.stsci.edu/hcasjobs
  [8]: https://github.com/rlwastro/mastcasjobs
  [9]: https://catalogs.mast.stsci.edu/docs/hsc.html
  [10]: ../HCV_API/HCV_API_demo.ipynb

# Instructions: 
* Complete the initialization steps [described below](#initialization).
* Run the notebook to completion. 
* Modify and rerun any sections of the Table of Contents below.

# Table of Contents
* [Intialization](#initialization)
* [Variable objects in IC 1613](#ic1613)
    * [Name resolver](#resolver)
    * [Select objects from HCV](#summary)
    * [Information on HCV variable classification](#classification)
    * [Sky coverage](#positions)
    * [Properties of variable objects](#variability)
    * [Color magnitude diagram](#cmd)
* [Light curve for a nova in M87](#m87)
    * [Extract and plot light curve for the nova](#lightcurve)
    * [HLA cutout images for selected measurements](#cutouts)
* [Compare the HCV automatic classification to expert validations](#expert)
    * [Plot MAD variability index distribution](#mad_expert)
    * [Plot fraction of artifacts vs. MAD](#artifacts)
* [Plot light curve for most variable high-quality candidate in the HCV](#most_variable)

# Initialization <a name="initialization"></a>

### Install Python modules

1. _This notebook requires the use of **Python 3**._
1. Modules can be installed with `conda`, if using the [Anaconda distribution][1] of python, or with `pip`.
   - If you are using `conda`, do not install / update / remove a module with `pip`, that exists in a `conda` [channel][1].
   - If a module is not available with `conda`, then it's okay to install it with `pip`
1. Install `mastcasjobs` and `casjobs` with `pip`:

```
pip install mastcasjobs
```

### Set up your CasJobs account information

You must have a [MAST Casjobs][3] account. Note that MAST Casjobs accounts are independent of SDSS Casjobs accounts.

For easy startup, you can optionally set the environment variables `CASJOBS_USERID` and/or `CASJOBS_PW` with your Casjobs account information.  The Casjobs user ID and password are what you enter when logging into Casjobs.

This script prompts for your Casjobs user ID and password during initialization if the environment variables are not defined.


  [1]: https://www.anaconda.com/products/distribution
  [2]: https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/channels.html
  [3]: https://mastweb.stsci.edu/hcasjobs

In [ ]:
import astropy
from astropy.coordinates import SkyCoord
import time
import sys
import os
import requests
import json
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from io import BytesIO

from astropy.table import Table, join

# check that version of mastcasjobs is new enough
# we are using some features not in version 0.0.1
from pkg_resources import get_distribution
from packaging.version import Version as V

assert V(get_distribution("mastcasjobs").version) >= V('0.0.2'), """
A newer version of mastcasjobs is required.
Update mastcasjobs to current version using this command:
pip install --upgrade git+git://github.com/rlwastro/mastcasjobs@master
"""

import mastcasjobs

# set width for pprint
astropy.conf.max_width = 150

In [ ]:
# set universal matplotlib parameters
plt.rcParams.update({'font.size': 16})

In [ ]:
HSCContext = "HSCv3"

Set up Casjobs environment.

In [ ]:
import getpass
if not os.environ.get('CASJOBS_USERID'):
    os.environ['CASJOBS_USERID'] = input('Enter Casjobs UserID:')
if not os.environ.get('CASJOBS_PW'):
    os.environ['CASJOBS_PW'] = getpass.getpass('Enter Casjobs password:')

## Variable objects near IC 1613 <a name="ic1613"></a>

### Use `astropy` name resolver to get position of IC 1613 <a name="resolver"></a>

In [ ]:
target = 'IC 1613'
coord_ic1613 = SkyCoord.from_name(target)

ra_ic1613 = coord_ic1613.ra.degree
dec_ic1613 = coord_ic1613.dec.degree
print(f'ra: {ra_ic1613}\ndec: {dec_ic1613}')

### Select objects near IC 1613 with ACS F475W and F814W measurements from HCV <a name="summary"></a>

This searches the HCV summary table for objects within 0.5 degrees of the galaxy center.  Note that this returns both variable and non-variable objects.  We restrict the sample to objects with measurements in the two filters of interest.  This uses the `SearchHCVMatchID` function to do the cone search.

In [ ]:
DBtable = "HCV_demo"
jobs = mastcasjobs.MastCasJobs(context="MyDB")

# drop table if it already exists
jobs.drop_table_if_exists(DBtable)

# get main information
radius = 1800.0 # arcsec
query = f"""
select m.MatchID, m.GroupID, m.SubGroupID, m.RA, m.Dec,
   m.AutoClass, m.ExpertClass, m.NumFilters,
   f.Filter, f.FilterDetFlag, f.VarQualFlag, f.NumLC, 
   f.MeanMag, f.MeanCorrMag, f.MAD, f.Chi2
into mydb.{DBtable}
from SearchHCVMatchID({ra_ic1613},{dec_ic1613},{radius}) s
join HCVmatch m on m.MatchID=s.MatchID
join HCVfilter f on f.MatchID=s.MatchID and (f.Filter='ACS_F475W' or f.Filter='ACS_F814W')
"""

t0 = time.time()
results = jobs.quick(query, task_name="HCV demo", context=HSCContext)

print(f"Completed in {(time.time()-t0):.1f} sec")
print(results)

# fast retrieval using special MAST Casjobs service
tab = jobs.fast_table(DBtable, verbose=True)

# clean up the output format
tab['MeanMag'].format = "{:.3f}"
tab['MeanCorrMag'].format = "{:.3f}"
tab['MAD'].format = "{:.4f}"
tab['Chi2'].format = "{:.4f}"
tab['RA'].format = "{:.6f}"
tab['Dec'].format = "{:.6f}"

# show some of the variable sources
tab[tab['AutoClass'] > 0]

### Description of the variable classification columns <a name="classification"></a>

Several of the table columns have information on the variability.

* The columns `AutoClass` and `ExpertClass` have summary information on the variability for a given `MatchID` object.
    * `AutoClass`: Classification as provided by the system: 0=constant 1=single filter variable candidate (SFVC) 2=multi-filter variable candidate (MFVC)
    * `ExpertClass`: Classification as provided by expert: 0=not classified by expert, 1=high confidence variable, 2=probable variable, 4=possible artifact
* The columns `MAD` and `Chi2` are variability indices using the median absolute deviation and the $\chi^2$ parameter for the given filter.
* The column `VarQualFlag` is a variability quality flag (see Section 5 of the paper). The five letters correspond to CI, D, MagerrAper2, MagAper2-MagAuto, p2p; AAAAA corresponds to the highest quality flag.
* The column `FilterDetFlag` is the filter detection flag: 1=source is variable in this filter, 0=source is not variable in this filter.

See the HCV paper by [Bonanos et al. (2019, AAp)](https://www.aanda.org/component/article?access=doi&doi=10.1051/0004-6361/201936026) for more details on the computation and meaning of these quantities.

### Find objects with measurements in both F475W and F814W

This could also be done in the SQL query.  Here we use the `Astropy.table.join` function instead.

In [ ]:
# the only key needed to do the join is MatchID, but we include other common columns
# so that join includes only one copy of them
jtab = join(tab[tab['Filter'] == 'ACS_F475W'], tab[tab['Filter'] == 'ACS_F814W'],
            keys=['MatchID', 'GroupID', 'SubGroupID', 'RA', 'Dec', 'AutoClass', 'ExpertClass'],
            table_names=['f475', 'f814'])
print(len(jtab), "matched F475W+F814W objects")
jtab[jtab['AutoClass'] > 0]

### Plot object positions on the sky <a name="positions"></a>

We mark the galaxy center as well.  Note that this field is in the outskirts of IC 1613.  The 0.5 degree search radius (which is the maximum allowed in the API) allows finding these objects.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot('RA', 'Dec', 'bo', markersize=1, label=f'{len(tab):,} HCV measurements', data=jtab)
ax.plot(ra_ic1613, dec_ic1613, 'rx', label=target, markersize=10)
ax.invert_xaxis()
ax.set(aspect='equal', xlabel='RA [deg]', ylabel='Dec [deg]')
ax.legend(loc='best')

### Plot HCV MAD variability index versus magnitude in F475W <a name="variability"></a>

The median absolute deviation variability index is used by the HCV to identify variables.  It measures the scatter among the multi-epoch measurements.  Some scatter is expected from noise (which increases for fainter objects).   Objects with MAD values that are high are likely to be variable.

This plots single-filter and multi-filter variable candidates (SFVC and MFVC) in different colors.  Note that variable objects with low F475W MAD values are variable in a different filter (typically F814W in this field).

This plot is similar to the upper panel of Figure 4 in [Bonanos et al. (2019, AAp)](https://www.aanda.org/component/article?access=doi&doi=10.1051/0004-6361/201936026).

In [ ]:
# define plot parameter lists
auto_class = np.unique(jtab['AutoClass'])
markers = ['x', 'o', 'o']
colors = ['silver', 'blue', 'tab:cyan']
labels = ['non-', 'single-filter ', 'multi-filter ']

fig, ax = plt.subplots(figsize=(15, 10))
for ac, marker, color, label in zip(auto_class, markers, colors, labels):
    data = jtab[jtab['AutoClass'] == ac]
    ax.plot('MeanCorrMag_f475', 'MAD_f475', marker, markersize=4, color=color,
            label=f'{len(data):,} {label}variable candidates', data=data)

ax.set(xlabel='ACS_F475W [mag]', ylabel='ACS_F475W MAD [mag]')
ax.legend(loc='best', title=f'{len(jtab)} HSC measurements near {target}')

### Plot variables in color-magnitude diagram <a name="cmd"></a>

Many of the candidate variables lie on the instability strip.

This plot is similar to the lower panel of Figure 4 in [Bonanos et al. (2019, AAp)](https://www.aanda.org/component/article?access=doi&doi=10.1051/0004-6361/201936026).

In [ ]:
# add a new column to jtab
jtab['MCMf475-MCMf814'] = jtab['MeanCorrMag_f475'] - jtab['MeanCorrMag_f814']

fig, ax = plt.subplots(figsize=(15, 10))
# uses same plot parameters defined in the previous plot
for ac, marker, color, label in zip(auto_class, markers, colors, labels):
    data = jtab[jtab['AutoClass'] == ac]
    ax.plot('MCMf475-MCMf814', 'MeanCorrMag_f475', marker, markersize=4, color=color,
            label=f'{len(data):,} {label}variable candidates', data=data)
    
ax.invert_yaxis()
ax.set(xlim=(-1.1, 4), xlabel='ACS F475W - F814W [mag]', ylabel='ACS F475W [mag]')
ax.legend(loc='best', title=f'{len(jtab)} HSC measurements near {target}')

## Get a light curve for a nova in M87 <a name="m87"></a>

### Extract light curve for a given MatchID <a name="lightcurve"></a>

Note that the `MatchID` could be determined by positional searches, filtering the catalog, etc.  This object comes from the top left panel of Figure 9 in [Bonanos et al. (2019, AAp)](https://www.aanda.org/component/article?access=doi&doi=10.1051/0004-6361/201936026).

In [ ]:
matchid = 1905457

jobs = mastcasjobs.MastCasJobs(context=HSCContext)
t0 = time.time()

# get light curves for F606W and F814W
nova_606 = jobs.quick(f"""select * from HCVdetailed
where MatchID={matchid} and Filter='ACS_F606W'""", task_name="HCV demo")
print(f"{(time.time()-t0):.1f} sec: retrieved {len(nova_606)} F606W measurements")

nova_814 = jobs.quick("""select * from HCVdetailed
where MatchID={} and Filter='ACS_F814W'
""".format(matchid), task_name="HCV demo")
print(f"{(time.time()-t0):.1f} sec: retrieved {len(nova_814)} F814W measurements")

# get the object RA and Dec as well
nova_tab = jobs.quick(f"""select MatchID, RA, Dec from HCVmatch
where MatchID={matchid}""", task_name="HCV demo")
print(f"{(time.time()-t0):.1f} sec: retrieved object info")

nova_606

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

ax.errorbar(x='MJD', y='CorrMag', yerr='MagErr', fmt='ob', ecolor='k', elinewidth=1, markersize=8, label='ACS F606W', data=nova_606)
ax.errorbar(x='MJD', y='CorrMag', yerr='MagErr', fmt='or', ecolor='k', elinewidth=1, markersize=8, label='ACS F814W', data=nova_814)

ax.invert_yaxis()
ax.set(xlabel='MJD [days]', ylabel='magnitude')
ax.legend(loc='best', title=f'Nova in M87 (MatchID: {matchid})')

### Get HLA image cutouts for the nova <a name="cutouts"></a>

The [Hubble Legacy Archive (HLA)](https://hla.stsci.edu) images were the source of the measurements in the HSC and HCV, and it can be useful to look at the images.  Examination of the images can be useful to identified cosmic-ray contamination and other possible image artifacts.  In this case, no issues are seen, so the light curve is reliable.

Note that the ACS F606W images of M87 have only a single exposure, so they do have cosmic ray contamination.  The accompanying F814W images have multiple exposures, allowing CRs to be removed.  In this case the F814W combined image is used to find objects, while the F606W exposure is used only for photometry.  That reduces the effects of F606W CRs on the catalog but it is still a good idea to confirm the quality of the images.

The `get_hla_cutout` function reads a single cutout image (as a JPEG grayscale image) and returns a PIL image object.  See the documentation on the [fitscut image cutout service](http://hla.stsci.edu/fitscutcgi_interface.html) for more information on the web service being used.

In [ ]:
def get_hla_cutout(imagename, ra, dec, size=33, autoscale=99.5, asinh=True, zoom=1):
    """Get JPEG cutout for an image"""
    url = "https://hla.stsci.edu/cgi-bin/fitscut.cgi"
    r = requests.get(url, params=dict(ra=ra, dec=dec, size=size, format="jpeg",
                                      red=imagename, autoscale=autoscale, asinh=asinh, zoom=zoom))
    im = Image.open(BytesIO(r.content))
    return im

In [ ]:
# sort images by magnitude from brightest to faintest
phot = nova_606
isort = np.argsort(phot['CorrMag'])
# select the brightest, median and faintest magnitudes
ind = [isort[0], isort[len(isort)//2], isort[-1]]

# we plot zoomed-in and zoomed-out views side-by-side for each selected image
nim = len(ind)*2
ncols = 2 # images per row
nrows = (nim+ncols-1)//ncols

imsize1 = 19
imsize2 = 101
mra = nova_tab['RA'][0]
mdec = nova_tab['Dec'][0]

# define figure and axes
fig, axes = plt.subplots(nrows, ncols, figsize=(12, (12/ncols)*nrows), tight_layout=True)

t0 = time.time()

# iterate through each set of two subplots in axes
for (ax1, ax2), k in zip(axes, ind):
    
    # get the images
    im1 = get_hla_cutout(phot['ImageName'][k], mra, mdec, size=imsize1)
    im2 = get_hla_cutout(phot['ImageName'][k], mra, mdec, size=imsize2)
    
    # plot left column   
    ax1.imshow(im1, origin="upper", cmap="gray")
    ax1.set_title(f"{phot['ImageName'][k]} m={phot['CorrMag'][k]:.3f}", fontsize=14)
    
    # plot right column
    ax2.imshow(im2, origin="upper", cmap="gray")
    xbox = np.array([-1, 1])*imsize1/2 + (imsize2-1)//2
    ax2.plot(xbox[[0, 1, 1, 0, 0]], xbox[[0, 0, 1, 1, 0]], 'r-', linewidth=1)
    ax2.set_title(f"{phot['ImageName'][k]} m={phot['CorrMag'][k]:.3f}", fontsize=14)    
    
print(f"{(time.time()-t0):.1f} s: got {nrows*ncols} cutouts")

## Compare the HCV automatic classification to expert validations <a name="expert"></a>

The HCV includes an automatic classification `AutoClass` for candidate variables as well as an expert validation for some fields that were selected for visual examination.  See the [description of the classification columns](#classification) and the HCV paper by [Bonanos et al. (2019, AAp)](https://www.aanda.org/component/article?access=doi&doi=10.1051/0004-6361/201936026) for more details on the computation and meaning of these quantities.

For this example, we select all the objects in the HCV that have expert classification information.

In [ ]:
DBtable = "HCV_demo2"
jobs = mastcasjobs.MastCasJobs(context="MyDB")

# drop table if it already exists
jobs.drop_table_if_exists(DBtable)

# get data for objects with an expert validation
query = f"""
select m.MatchID, m.GroupID, m.SubGroupID, m.RA, m.Dec,
   m.AutoClass, m.ExpertClass, m.NumFilters,
   f.Filter, f.FilterDetFlag, f.VarQualFlag, f.NumLC, 
   f.MeanMag, f.MeanCorrMag, f.MAD, f.Chi2
into mydb.{DBtable}
from HCVmatch m
join HCVfilter f on m.MatchID=f.MatchID
where m.ExpertClass>0
"""

t0 = time.time()
results = jobs.quick(query, task_name="HCV demo", context=HSCContext)

print(f"Completed in {(time.time()-t0):.1f} sec")
print(results)

# fast retrieval using special MAST Casjobs service
tab = jobs.fast_table(DBtable, verbose=True)

# clean up the output format
tab['MeanMag'].format = "{:.3f}"
tab['MeanCorrMag'].format = "{:.3f}"
tab['MAD'].format = "{:.4f}"
tab['Chi2'].format = "{:.4f}"
tab['RA'].format = "{:.6f}"
tab['Dec'].format = "{:.6f}"

# tab includes 1 row for each filter (so multiple rows for objects with multiple filters)
# get an array that has only one row per object
mval, uindex = np.unique(tab['MatchID'], return_index=True)
utab = tab[uindex]
print(f"{len(utab)} unique MatchIDs in table")

tab

An `ExpertClass` value of 1 indicates that the object is confidently confirmed to be a variable; 2 means that the measurements do not have apparent problems and so the object is likely to be variable (usually the variability is too small to be obvious in the image); 4 means that the variability is likely to be the result of artifacts in the image (e.g., residual cosmic rays or diffraction spikes from nearby bright stars).

Compare the distributions for single-filter variable candidates (SFVC, `AutoClass`=1) and multi-filter variable candidates (MFVC, `AutoClass`=2).  The fraction of artifacts is lower in the MFVC sample.

In [ ]:
sfcount = np.bincount(utab['ExpertClass'][utab['AutoClass'] == 1])
mfcount = np.bincount(utab['ExpertClass'][utab['AutoClass'] == 2])
sfrat = sfcount/sfcount.sum()
mfrat = mfcount/mfcount.sum()

print("Type Variable Likely Artifact Total")
print("SFVC {:8d} {:6d} {:8d} {:5d} counts".format(sfcount[1], sfcount[2], sfcount[4], sfcount.sum()))
print("MFVC {:8d} {:6d} {:8d} {:5d} counts".format(mfcount[1], mfcount[2], mfcount[4], mfcount.sum()))
print("SFVC {:8.3f} {:6.3f} {:8.3f} {:5.3f} fraction".format(sfrat[1], sfrat[2], sfrat[4], sfrat.sum()))
print("MFVC {:8.3f} {:6.3f} {:8.3f} {:5.3f} fraction".format(mfrat[1], mfrat[2], mfrat[4], mfrat.sum()))

### Plot the MAD variability index distribution with expert classifications <a name="mad_expert"></a>

Note that only the filters identified as variable (`FilterDetFlag` > 0) are included here.

This version of the plot shows the distributions for the various `ExpertClass` values along with, for comparison, the distribution for all objects in gray (which is identical in each panel).  Most objects are classified as confident or likely variables.  Objects with lower MAD values (indicating a lower amplitude of variability) are less likely to be identified as confident variables because low-level variability is more difficult to confirm via visual examination.

#### Data & Bins

In [ ]:
w = np.where(tab['FilterDetFlag'] > 0)
mad = tab['MAD'][w]
e = tab['ExpertClass'][w]

xrange = [7.e-3, 2.0]
bins = xrange[0]*(xrange[1]/xrange[0])**np.linspace(0.0, 1.0, 50)

#### Plots

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 12))

labels = ['Confident', 'Likely', 'Artifact']
colors = ['C2', 'C1', 'C0']

for ax, v, label, color in zip(axes, np.unique(e), labels, colors):
    ax.hist(mad, bins=bins, log=True, color='lightgray', label='All')
    ax.hist(mad[e == v], bins=bins, log=True, label=label, color=color)
    ax.set(xscale='log', ylabel='Count')
    ax.legend(loc='upper left')
    
fig.suptitle('HCV Expert Validation', y=0.9)
_ = axes[2].set_xlabel('MAD Variability Index [mag]')

The plot below shows the same distributions, but plotted as stacked histograms.  The top panel uses a linear scale on the y-axis and the bottom panel uses a log y scale.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 12))

ylogs = [False, True]

for ax, ylog in zip(axes, ylogs):
    ax.hist(mad, bins=bins, log=ylog, label='Artifact')
    ax.hist(mad[e < 4], bins=bins, log=ylog, label='Likely Variable')
    ax.hist(mad[e == 1], bins=bins, log=ylog, label='Confident Variable')

    ax.set_xscale('log')
    ax.set_xlabel('MAD Variability Index [mag]')
    ax.set_ylabel('Count')
    ax.legend(loc='upper right', title='HCV Expert Validation')

### Plot the fraction of artifacts as a function of MAD variability index <a name="artifacts"></a>

This shows how the fraction of artifacts varies with the MAD value.  For larger MAD values the fraction decreases sharply, presumably because such large values are less likely to result from the usual artifacts.  Interestingly, the artifact fraction also declines for smaller MAD values (MAD < 0.1 mag).  Probably that happens because typical artifacts are more likely to produce strong signals than the weaker signals indicated by a low MAD value. 

In [ ]:
w = np.where(tab['FilterDetFlag'] > 0)
mad = tab['MAD'][w]
e = tab['ExpertClass'][w]

xrange = [7.e-3, 2.0]
bins = xrange[0]*(xrange[1]/xrange[0])**np.linspace(0.0, 1.0, 30)

all_count, bin_edges = np.histogram(mad, bins=bins)
artifact_count, bin_edges = np.histogram(mad[e == 4], bins=bins)
wnz = np.where(all_count > 0)[0]
nnz = len(wnz)

artifact_count = artifact_count[wnz]
all_count = all_count[wnz]
xerr = np.empty((2, nnz), dtype=float)
xerr[0] = bin_edges[wnz]
xerr[1] = bin_edges[wnz+1]

# combine bins at edge into one big bin to improve the statistics there
iz = np.where(all_count.cumsum() > 10)[0][0]
if iz > 0:
    all_count[iz] += all_count[:iz].sum()
    artifact_count[iz] += artifact_count[:iz].sum()
    xerr[0, iz] = xerr[0, 0]
    all_count = all_count[iz:]
    artifact_count = artifact_count[iz:]
    xerr = xerr[:, iz:]
    
iz = np.where(all_count[::-1].cumsum() > 40)[0][0]
if iz > 0:
    all_count[-iz-1] += all_count[-iz:].sum()
    artifact_count[-iz-1] = artifact_count[-iz:].sum()
    xerr[1, -iz-1] = xerr[1, -1]
    all_count = all_count[:-iz]
    artifact_count = artifact_count[:-iz]
    xerr = xerr[:, :-iz]

x = np.sqrt(xerr[0]*xerr[1])
xerr[0] = x - xerr[0]
xerr[1] = xerr[1] - x

frac = artifact_count/all_count
# error on fraction using binomial distribution (approximate)
ferr = np.sqrt(frac*(1-frac)/all_count)

Create the plot

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
ax.errorbar(x, frac, xerr=xerr, yerr=ferr, fmt='ob', markersize=5, label='Artifact fraction')
ax.set(xscale='log', xlabel='MAD Variability Index [mag]', ylabel='Artifact Fraction')
ax.legend(loc='upper right', title='HCV Expert Validation')

### Plot light curve for the most variable high quality candidate in the HCV <a name="most_variable"></a>

Select the candidate variable with the largest MAD value and `VarQualFlag` = 'AAAAA'.  To find the highest MAD value, we sort by MAD in descending order and select the first result.

In [ ]:
jobs = mastcasjobs.MastCasJobs(context=HSCContext)

# join to the Groups table as well to get the target name

query = """
select top 1 m.MatchID, m.GroupID, m.SubGroupID, g.TargetName, m.RA, m.Dec,
   m.AutoClass, m.ExpertClass, m.NumFilters,
   f.Filter, f.FilterDetFlag, f.VarQualFlag, f.NumLC, 
   f.MeanMag, f.MeanCorrMag, f.MAD, f.Chi2
from HCVmatch m
join HCVfilter f on m.MatchID=f.MatchID
join Groups g on m.GroupID=g.GroupID
where f.VarQualFlag='AAAAA'
order by f.MAD desc
"""

t0 = time.time()
tab = jobs.quick(query, task_name="HCV demo", context=HSCContext)

print(f"Completed in {(time.time()-t0):.1f} sec")

# clean up the output format
tab['MeanMag'].format = "{:.3f}"
tab['MeanCorrMag'].format = "{:.3f}"
tab['MAD'].format = "{:.4f}"
tab['Chi2'].format = "{:.4f}"
tab['RA'].format = "{:.6f}"
tab['Dec'].format = "{:.6f}"

print(f"MatchID {tab['MatchID'][0]} in group '{tab['TargetName'][0]}' has largest MAD value = {tab['MAD'][0]:.2f}")
tab

Get the light curve.

In [ ]:
matchid = tab['MatchID'][0]
mfilter = tab['Filter'][0]

jobs = mastcasjobs.MastCasJobs(context=HSCContext)
t0 = time.time()

# get light curves for F606W and F814W
lc = jobs.quick("""select * from HCVdetailed
where MatchID={} and Filter='{}'
""".format(matchid, mfilter), task_name="HCV demo")
print(f"{(time.time()-t0):.1f} sec: retrieved {len(lc)} {mfilter} measurements")

Plot the light curve.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))

ax.errorbar(x='MJD', y='CorrMag', yerr='MagErr', fmt='ob', ecolor='k', elinewidth=1, markersize=8, label=mfilter, data=lc)

ax.invert_yaxis()
ax.set(xlabel='MJD [days]', ylabel='magnitude')
ax.legend(loc='best', title=f"MatchID: {matchid} in {tab['TargetName'][0]} MAD={tab['MAD'][0]:.2f}")

Extract cutout images for the entire light curve (since it does not have many points).

In [ ]:
# sort images in MJD order
ind = np.argsort(lc['MJD'])

# we plot zoomed-in and zoomed-out views side-by-side for each selected image
nim = len(ind)*2
ncols = 2 # images per row
nrows = (nim+ncols-1)//ncols

imsize1 = 19
imsize2 = 101
mra = tab['RA'][0]
mdec = tab['Dec'][0]

# define figure and axes
fig, axes = plt.subplots(nrows, ncols, figsize=(12, (12/ncols)*nrows), tight_layout=True)

t0 = time.time()

# iterate through each set of two subplots in axes
for i, ((ax1, ax2), k) in enumerate(zip(axes, ind), 1):
    
    # get the images
    im1 = get_hla_cutout(lc['ImageName'][k], mra, mdec, size=imsize1)
    im2 = get_hla_cutout(lc['ImageName'][k], mra, mdec, size=imsize2)
    
    # plot left column    
    ax1.imshow(im1, origin="upper", cmap="gray")
    ax1.set_title(lc['ImageName'][k], fontsize=14)
    
    # plot right column
    ax2.imshow(im2, origin="upper", cmap="gray")
    xbox = np.array([-1, 1])*imsize1/2 + (imsize2-1)//2
    ax2.plot(xbox[[0, 1, 1, 0, 0]], xbox[[0, 0, 1, 1, 0]], 'r-', linewidth=1)
    ax2.set_title(f"m={lc['CorrMag'][k]:.3f} MJD={lc['MJD'][k]:.2f}", fontsize=14)
    
    print(f"{(time.time()-t0):.1f} s: finished {i} of {len(ind)} epochs")
    
print(f"{(time.time()-t0):.1f} s: got {nrows*ncols} cutouts")